# Description

This script's purpose is to create a unique identifier (id) for each pixel in the image. 

This is done by transforming the pixels of an image into an array and assigning a new unique value to each array element. 
This new array is then used to populate a new .tif file in which each image pixel will be a unique identifier for that pixel

In [ ]:
from osgeo import gdal
from affine import Affine
import numpy as np
import rasterio

In [ ]:
#Reading the image file and obtaining its existing  coordinates information
step1 = gdal.Open('00/20190312/B01_20190312', gdal.GA_ReadOnly)
GT_input = step1.GetGeoTransform()
print(GT_input)
transf = (399664.5859337867, 10.0, 0.0, 1100650.3206286626, 0.0, -10.0) #as obtained from GT_input
afn = Affine.from_gdal(*transf)

In [ ]:
#Reading the image and creating an array from it 
step2 = step1.GetRasterBand(1)
img_as_array = step2.ReadAsArray()
x =img_as_array
size1,size2=img_as_array.shape
output=np.zeros(shape=(size1,size2))
print("Image shape is",output.shape)

x_flatten = x.flatten()
print("Flattened image shape is",x_flatten.shape)

In [ ]:
#Creating the unique ids
Field_id = np.arange(len(x_flatten))
result = np.column_stack((x_flatten,Field_id))
print ("resultant array", str(result))

#Keeping only the newly created id column and reshaping it to the original shape
result_deleted = np.delete(result, 0, 1)
print ("resultant array after deleting", str(result_deleted))
array_output = np.reshape(result_deleted, (11021, 11125))#enter details as obtained from output.shape

In [ ]:
#Exporting the array created as a tif file
dst_crs='EPSG:32630'#coordinate system of the destination tif
output = np.int32(array_output)

In [ ]:
with rasterio.open(
    'Fieldid.tif',#name of the outputfile 
    'w',
    driver='GTiff',
    height=output.shape[0],
    width=output.shape[1],
    count=1,
    dtype=np.int32,
    crs=dst_crs,
    transform=afn,
) as dest_file:
    dest_file.write(output, 1)
dest_file.close()